In [7]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

hm_epochs = 10
n_classes = 10
batch_size = 128

chunk_size = 28
n_chunks = 28
rnn_size = 128

x = tf.placeholder('float',[None,n_chunks,chunk_size])
y = tf.placeholder('float')

def neural_network_model(x):
    
    #(input_data * weights) + bias
    
    layer = {'weights': tf.Variable(tf.random_normal([rnn_size,n_classes])),
                     'biases':tf.Variable(tf.random_normal([n_classes]))}
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1,chunk_size])
    x = tf.split(x, n_chunks, 0)
    
    lstm_cell = rnn.BasicLSTMCell(rnn_size)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']
    
    return output

def train_nueral_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    #cycles of feed forward + backprop

    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))
                
                _, c = sess.run([optimizer,cost],feed_dict= {x:epoch_x,y:epoch_y})
                epoch_loss += c
            print('Epoch',epoch+1,'completed out of', hm_epochs,'loss:',epoch_loss)
            
            
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1,n_chunks,chunk_size)),
                                         y:mnist.test.labels}))

        
train_nueral_network(x)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Epoch 0 completed out of 10 loss: 226.975723382
Epoch 1 completed out of 10 loss: 62.5504076686
Epoch 2 completed out of 10 loss: 41.3270212375
Epoch 3 completed out of 10 loss: 30.7257655398
Epoch 4 completed out of 10 loss: 26.0671058577
Epoch 5 completed out of 10 loss: 21.8314915546
Epoch 6 completed out of 10 loss: 17.9144906839
Epoch 7 completed out of 10 loss: 16.4774448057
Epoch 8 completed out of 10 loss: 13.9791191828
Epoch 9 completed out of 10 loss: 12.6073158034
Accuracy: 0.9836
